In [1]:
import modin.pandas as pd

from modin.config import NPartitions
print("Modin partitions:", NPartitions.get())

from modin.config import ProgressBar
from tqdm import tqdm
ProgressBar.enable()

from shapely.geometry import Polygon

Modin partitions: 28


In [2]:
places = pd.read_json("places.jsonl", lines=True)

To remove this warning, run the following python code before doing dataframe operations:

    import ray
    ray.init()

2023-07-19 11:21:56,940	INFO worker.py:1636 -- Started a local Ray instance.


In [3]:
# Convert 16-digit hex ids to 64 bit uints.
places["id"] = places["id"].apply(int, base=16).astype("uint64")
places.set_index("id", inplace=True)

Estimated completion of line 2:   0%           Elapsed time: 00:00, estimated remaining time: ?

In [4]:
# Drop certainly useless columns.
places.drop(columns=["url", "name", "country"], inplace=True)

In [5]:
# Ensure `attributes` and `contained_within` are empty, then drop them.
assert ~(places["attributes"].map(len) > 0).any()
assert ~(places["contained_within"].map(len) > 0).any()
places.drop(columns=["attributes", "contained_within"], inplace=True)

Estimated completion of line 2:   0%           Elapsed time: 00:00, estimated remaining time: ?

Estimated completion of line 3:   0%           Elapsed time: 00:00, estimated remaining time: ?

In [6]:
# Ensure `bounding_box`, where present, is a polygon with 4 vertices, then simplify the representation.
assert places["bounding_box"].map(lambda bb: bb["type"] == "Polygon").all()
assert places["bounding_box"].map(lambda bb: len(bb["coordinates"]) == 1).all()
assert (
    places["bounding_box"]
    .map(lambda bb: [len(p) for p in bb["coordinates"][0]] == [2, 2, 2, 2])
    .all()
)

places["bounding_box"] = places["bounding_box"].map(
    lambda bb: Polygon([tuple(point) for point in bb["coordinates"][0]]).wkt
)

Estimated completion of line 2:   0%           Elapsed time: 00:00, estimated remaining time: ?

Estimated completion of line 3:   0%           Elapsed time: 00:00, estimated remaining time: ?

Estimated completion of line 6:   0%           Elapsed time: 00:00, estimated remaining time: ?

Estimated completion of line 7:   0%           Elapsed time: 00:00, estimated remaining time: ?

In [7]:
# Dump to parquet. The dataset is tiny, so we force writing a single file.
places._to_pandas().to_parquet("places.parquet")

Estimated completion of line 10:   0%           Elapsed time: 00:00, estimated remaining time: ?